### Install relevant libraries and packages

In [1]:
import pandas as pd
import torch
from torch.nn import Linear
from torch.nn import Parameter
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from pytorch_metric_learning import losses, distances, reducers, testers
#from pcgrad import PCGrad
from tqdm import tqdm

###### ******* LOAD  COCRYSTAL GRAPG DATA *******

In [2]:
dataset = torch.load('mordred_graph_data.pt')
dataset

Data(x=[740, 3226], edge_index=[2, 1480], y=[740], edge_type=[1480])

### Initialize RGCNConv model 

In [3]:
class Net(torch.nn.Module):
    def __init__(self, dim1, dim2, dim3, dropout):
        super(Net, self).__init__()
        self.gene_emb = Parameter(torch.randn(740, 3226))
        self.conv1 = RGCNConv(3226, dim1, 4)
#         self.conv2 = RGCNConv(dim1, dim2, 4)
        self.lin1 = Linear(dim1, dim2)
        self.lin2 = Linear(dim2, 2)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_type):
        x = torch.cat((x, self.gene_emb), dim=0)
        x = F.relu(self.conv1(x, edge_index, edge_type))
        x = F.dropout(x, self.dropout, training=self.training)
#         x = F.relu(self.conv2(x, edge_index, edge_type))
#         x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.lin1(x))
        emb = x
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.lin2(x)

        return F.log_softmax(x, dim=-1), emb

#### Define Model for training

In [4]:
def model_train(train_idx):
    model.train()
    optimizer.zero_grad()
    out, emb = model(data.x, data.edge_index, data.edge_type)
    print('The out: ', out)
    print('The emb: ', emb)
    loss_tri = Loss_triplet(emb[train_idx], data.y[train_idx])
    loss_nll = F.nll_loss(out[train_idx], data.y[train_idx]).backward()
    loss_list = [loss_tri, loss_nll]
    #loss_list.backward()
    optimizer.step()

#### Check the device

In [5]:
# Re-introducing the device just incase you have access on the cuda GPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### Define model for evaluation

In [6]:
@torch.no_grad()
def model_val(val_idx, plus_test=False):
    model.eval()
    out, emb = model(data.x, data.edge_index, data.edge_type)
    out_val = out.exp()[val_idx]
    pred_val = out_val.argmax(dim=-1)
    auroc_val = roc_auc_score(data.y[val_idx].cpu(), out_val[:, 1].cpu())
    
    if plus_test == False:
        acc_val = accuracy_score(data.y[val_idx].cpu(), pred_val.cpu())
        auprc_val = average_precision_score(
                data.y[val_idx].cpu(), out_val[:, 0].cpu(), pos_label=0)
        sens_val = recall_score(data.y[val_idx].cpu(), pred_val.cpu())
        spec_val = recall_score(data.y[val_idx].cpu(), pred_val.cpu(), pos_label=0)
        mcc_val = matthews_corrcoef(data.y[val_idx].cpu(), pred_val.cpu())
        return acc_val, sens_val, spec_val, mcc_val, auroc_val, auprc_val
    
    else:
        out_test = out.exp()[test_index]
        pred_test = out_test.argmax(dim=-1)
        acc_test = accuracy_score(data.y[test_index].cpu(), pred_test.cpu())
        auroc_test = roc_auc_score(data.y[test_index].cpu(), out_test[:, 1].cpu())
        auprc_test = average_precision_score(
                data.y[test_index].cpu(), out_test[:, 0].cpu(), pos_label=0)
        sens_test = recall_score(data.y[test_index].cpu(), pred_test.cpu())
        spec_test = recall_score(data.y[test_index].cpu(), pred_test.cpu(), pos_label=0)
        mcc_test = matthews_corrcoef(data.y[test_index].cpu(), pred_test.cpu())
        return auroc_val, acc_test, sens_test, spec_test, mcc_test, auroc_test, auprc_test

###### ******* RECHECK  COVID-19 GRAPH DATA *******

In [7]:
dataset

Data(x=[740, 3226], edge_index=[2, 1480], y=[740], edge_type=[1480])

In [8]:
len(dataset)

4

In [9]:
data = dataset # Tensor representation of the Cocrstal data
print('covid_drugs_to_target_organism:', data)

covid_drugs_to_target_organism: Data(x=[740, 3226], edge_index=[2, 1480], y=[740], edge_type=[1480])


In [10]:
len(data)

4

In [11]:
data.num_features

3226

In [12]:
data.y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [13]:
data.y.unique()

tensor([0, 1])

In [14]:
data.y.unique

<bound method Tensor.unique of tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0

In [15]:
len(data.y.unique())

2

In [16]:
# Load data into cuda dataset
data = data.to(device)
data

Data(x=[740, 3226], edge_index=[2, 1480], y=[740], edge_type=[1480])

### Shuffling, Data split, and Kfold cross validation

###### Train and test data split initially

In [17]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=661)
sss

StratifiedShuffleSplit(n_splits=1, random_state=661, test_size=0.15,
            train_size=None)

In [18]:
train_index, test_index = next(sss.split(data.x.cpu(), data.y.cpu()))

In [19]:
train_index

array([ 59, 535, 235, 639, 575,  25, 472, 254, 513, 580, 506, 108, 638,
       719, 455, 543, 512, 198, 686, 481, 394, 439, 398, 143, 207, 679,
        89, 739, 395, 684, 660, 205, 404, 416, 256, 495, 166,  19, 268,
        66, 712, 402, 294, 607, 530, 258, 290, 531, 221, 134, 626, 285,
       714,   4,  72, 209, 598,  21,  41,  55, 365,  68, 555, 423, 161,
       447, 694, 692, 300, 318, 128, 240, 637, 737, 581, 471, 572, 484,
       403, 217, 346, 561, 255, 538, 247, 163, 590,  43, 587, 141, 690,
       528, 738, 241,  98, 529, 702,  71,   6, 704, 381, 523, 276,  86,
         8, 400,  88, 588, 685, 386, 364, 467,  30, 668,  10,  78,  39,
       180, 114, 218, 478, 532,  15, 722, 253, 474, 159, 171, 734, 200,
       459, 377, 696, 706, 248,  69, 133, 526, 713, 541, 408, 152, 288,
       454, 591, 178, 260, 616, 138, 601, 606, 689, 413, 307, 120, 328,
       194, 687, 597, 537, 146, 278, 341, 525,  38, 226, 515, 415, 220,
        63, 354,  57,  42, 488, 187, 263, 319, 453, 733, 682, 43

In [20]:
len(train_index)

629

In [21]:
test_index

array([608, 179, 110, 726, 375, 169, 594, 659,  83, 292, 125, 720, 489,
       468, 396, 674, 103, 336, 476, 458, 466, 366, 287,  35, 233, 257,
       723, 633, 349, 320, 578, 210, 426, 553, 188, 101, 244, 279, 558,
       245, 329, 444, 224, 170, 174, 184, 595, 119,  13, 231, 353,  77,
        85, 339, 440, 183, 605, 618, 412, 261, 678, 196, 109,   3, 160,
        49, 391, 554, 619, 729, 229, 655, 446, 308,  16, 409, 105, 465,
       211, 612, 111,  91,  27, 368, 641, 282, 262, 469, 482, 275, 189,
       150, 583, 363, 552, 420, 646, 107, 314, 611, 360, 367, 431, 192,
       651, 479, 705, 185, 136,  12, 158])

In [22]:
len(test_index)

111

##### Train and validation data split

In [23]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15/0.85, random_state=154)
sss

StratifiedShuffleSplit(n_splits=1, random_state=154,
            test_size=0.17647058823529413, train_size=None)

In [24]:
train_slice, val_slice = next(sss.split(data.x[train_index].cpu(), data.y[train_index].cpu()))

In [25]:
train_slice

array([204, 117, 416,  74, 244, 502, 529, 233, 206, 323,  73, 622, 157,
       615, 445, 621, 181, 557, 313, 143,  95,  48, 443, 468, 129, 123,
       249, 365,  98, 319, 377, 360,  88, 454,  18, 359, 186,  12, 486,
       243, 381, 246,  35, 515, 453, 138, 590,  29, 466,   8, 480, 259,
       321, 222, 178, 446, 613, 540,  84, 405, 440,  62, 537, 485, 192,
       126, 501, 458, 518, 482, 119, 324, 492, 560, 103, 265,  66, 426,
        33, 520, 550, 489, 358, 318, 627, 370, 465, 373, 279, 432, 109,
       113, 462,   4, 255, 242, 374, 538, 378, 450,  92, 276, 196, 341,
       451, 447, 301, 372, 457, 163, 198,   1,  27, 592, 425, 346,  41,
       471, 174,  71, 351, 283, 456, 337, 626, 235, 167, 314, 415, 585,
       170, 122, 331, 524, 221, 343, 477, 412, 407, 227, 141, 541, 580,
       384, 250, 162, 326, 579, 232,  86, 547,  14,  80, 185, 438, 555,
       164, 559, 380,  53, 487, 125, 315, 439, 532, 317, 536, 367, 612,
       183,  11, 203, 553, 421, 187, 236, 402, 542, 332, 202, 60

In [26]:
len(train_slice)

517

In [27]:
val_slice

array([571, 228, 581, 488, 175, 544, 409, 363, 200, 272,  42, 496, 430,
       339,   2, 508, 572,   7, 127, 172, 108, 497,  45, 411, 306, 589,
       609, 514, 493, 517, 441, 110, 154, 189, 410, 155,  47,  28, 511,
       564, 494, 254, 470, 625, 264, 575, 513, 519, 543, 177, 605, 404,
       302,  58, 509, 121, 490, 472, 437, 467,  50, 413, 132, 134, 601,
       574, 417, 152, 182, 274,  26, 399, 361, 388, 217, 237, 368,  97,
       219, 149, 463, 423, 295, 116, 362, 455, 312, 479, 355,  17, 512,
       271, 270, 398, 205,  59, 320, 224,  54, 208, 584,  51, 275, 527,
       356, 603, 350, 586,  68,  43, 469, 573])

In [28]:
len(val_slice)

112

In [29]:
val_index = train_index[val_slice]
val_index

array([727, 615, 671, 579, 263,  37,  18, 242, 113, 104, 294, 274, 338,
       604, 235,  96, 151, 254, 171,  42, 685, 475, 258, 518, 317, 334,
       310, 315, 557, 564, 701, 364, 120, 680, 142, 328, 531, 395,  36,
       340, 499, 624,  28, 432, 519, 259, 422, 243, 647, 453, 130,  87,
       345,  41, 193, 532, 693, 524, 272, 710, 626, 149, 696, 248,  11,
       442, 485, 413, 145, 725,  89,  32, 238, 172, 456, 718, 352,  71,
       124, 601, 568, 648, 664,  39, 289, 225,  56, 715, 384, 198, 536,
       492, 424, 313,  82,  55, 510, 355,  72,  31, 620, 285, 636, 534,
       460, 131, 717, 405, 300, 607, 491, 304])

In [30]:
len(val_index)

112

In [31]:
train_index = train_index[train_slice]
train_index

array([436, 180, 298, 581, 462, 625, 487, 335, 332, 599, 737, 311, 687,
       121, 542, 520, 204, 316, 337, 454, 529, 221, 707, 252, 200, 722,
       401, 388,   6, 333, 663,  50, 587, 593, 686, 249, 100, 638, 322,
       202, 503, 551, 495, 284, 505, 713, 299, 684, 418, 513, 643, 156,
       430, 246, 733, 182, 574, 494, 247, 708, 560, 555, 411, 451, 406,
       159, 154, 438, 545, 387, 218, 213, 357,  52,  86, 216, 694, 628,
       416, 621, 577, 206,  97, 372, 667, 173, 570,  95,  45, 350, 386,
       668, 303, 575,  34, 297, 266, 306, 425, 609, 738, 571, 374,  79,
       675, 699,  53, 271, 517, 525, 427, 535, 739, 548, 613, 407, 402,
       281, 187, 240, 731, 473, 508, 137,  46,  90, 415,  26, 445, 295,
       354,  15, 283,  47, 135, 457, 511, 709, 448, 203, 152, 698, 644,
       176, 452, 341, 177, 378, 650, 590, 450, 455, 346, 123, 127, 392,
        38, 239,  94,   4, 563, 474, 106, 691, 681, 312, 343, 157, 230,
       490, 108, 550, 117, 582, 305,  48, 652, 164, 376, 700, 59

In [32]:
len(train_index)

517

##### 10_fold cross validation

In [33]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=18)
skf

StratifiedKFold(n_splits=10, random_state=18, shuffle=True)

In [34]:
# Hyperparameters
lr = 0.00195
weight_decay = 0.00738
margin = 0.274
dropout = 0.645
triplets_per_anchor = 60
dim1 = 1340
dim2 = 920
dim3 = 740
low = 0.274

In [35]:
# triplet margin loss
distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=low)
Loss_triplet = losses.TripletMarginLoss(margin=margin, distance=distance, reducer=reducer,
                                triplets_per_anchor=triplets_per_anchor)

In [36]:
distance

CosineSimilarity()

In [37]:
reducer

ThresholdReducer()

In [38]:
Loss_triplet

TripletMarginLoss(
  (distance): CosineSimilarity()
  (reducer): ThresholdReducer()
)

In [39]:
# cross-validation
acc_list = []
sens_list = []
spec_list = []
mcc_list = []
auroc_list = []
auprc_list = []
epoch_list = []

In [50]:
print('Cross-validation progressing...')
for train_mask, val_mask in tqdm(skf.split(data.x[train_index].cpu(), data.y[train_index].cpu())):
    model = Net(dim1, dim2, dim3, dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    #optimizer = PCGrad(torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay))

    auroc_max = 0
    epoch_count = 0
    for epoch in range(500):
        epoch_count += 1
        model_train(train_index[train_mask])
        acc, sens, spec, mcc, auroc, auprc = model_val(train_index[val_mask])
        if auroc > auroc_max:
            epoch_count = 0
            acc_max = acc
            auroc_max = auroc
            auprc_max = auprc
            sens_max = sens
            spec_max = spec
            mcc_max = mcc
            epoch_max = epoch
        if epoch_count == 30:
            break

    acc_list.append(acc_max)
    sens_list.append(sens_max)
    spec_list.append(spec_max)
    mcc_list.append(mcc_max)
    auroc_list.append(auroc_max)
    auprc_list.append(auprc_max)
    epoch_list.append(epoch_max)

Cross-validation progressing...


0it [00:00, ?it/s]

The out:  tensor([[-0.6342, -0.7558],
        [-0.9674, -0.4782],
        [-0.6174, -0.7751],
        ...,
        [-0.6539, -0.7340],
        [-0.3267, -1.2775],
        [-0.8069, -0.5910]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.2502, 0.2971,  ..., 0.6029, 0.0000, 0.0000],
        [0.0000, 0.5991, 0.0000,  ..., 1.9227, 0.7859, 0.0000],
        [0.4437, 0.9028, 0.7403,  ..., 0.9074, 0.0802, 0.0000],
        ...,
        [0.4529, 1.1371, 0.0000,  ..., 1.3049, 0.0000, 1.1282],
        [0.9914, 0.2006, 0.0000,  ..., 0.1320, 0.2283, 0.5785],
        [0.0000, 1.1359, 0.0190,  ..., 0.0000, 0.5448, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-2.4235e+00, -9.2788e-02],
        [-3.8712e-04, -7.8570e+00],
        [-3.7806e-04, -7.8807e+00],
        ...,
        [-1.7204e+00, -1.9723e-01],
        [-2.7335e+00, -6.7202e-02],
        [-3.1127e+00, -4.5503e-02]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:

The out:  tensor([[-1.0015, -0.4578],
        [-0.0664, -2.7451],
        [-0.1645, -1.8860],
        ...,
        [-0.9297, -0.5020],
        [-0.6936, -0.6927],
        [-0.7831, -0.6106]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ReluBackward0>)
The out:  tensor([[-1.2967, -0.3194],
        [-0.1608, -1.9068],
        [-0.0358, -3.3471],
        ...,
        [-0.7353, -0.6527],
        [-0.7464, -0.6426],
        [-0.9627, -0.4810]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..

The out:  tensor([[-0.6250, -0.7663],
        [-0.0215, -3.8513],
        [-0.0054, -5.2282],
        ...,
        [-0.6272, -0.7638],
        [-0.6943, -0.6920],
        [-1.2533, -0.3363]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ReluBackward0>)
The out:  tensor([[-0.6833, -0.7031],
        [-0.0307, -3.5001],
        [-0.1177, -2.1982],
        ...,
        [-0.6710, -0.7158],
        [-0.6232, -0.7683],
        [-1.1339, -0.3883]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..

...

The out:  tensor([[-0.7766, -0.6162],
        [-0.0087, -4.7440],
        [-0.0146, -4.2320],
        ...,
        [-1.0349, -0.4389],
        [-0.5015, -0.9304],
        [-0.5064, -0.9230]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5341, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5996, 0.0000],
        [0.0000, 0.0169, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-2.0432e-01, -1.6885e+00],
        [-8.0720e-04, -7.1223e+00],
        [-1.0011e-02, -4.6091e+00],
        ...,
        [-9.4512e-01, -4.9206e-01],
        [-5.4564e-01, -8.6624e-01],
        [-4.9753e-01, -9.3657e-01]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:

The out:  tensor([[-1.2117e+00, -3.5336e-01],
        [-5.1757e-02, -2.9870e+00],
        [-1.6327e-03, -6.4184e+00],
        ...,
        [-1.4481e+00, -2.6790e-01],
        [-7.7990e-01, -6.1332e-01],
        [-4.6074e-01, -9.9646e-01]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.3635, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2103, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4880, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0182, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0151, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0944, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-2.7906e-02, -3.5928e+00],
        [-4.6949e-03, -5.3636e+00],
        [-2.4816e-04, -8.3015e+00],
        ...,
        [-1.7820e+00, -1.8429e-01],
        [-7.2519e-01, -6.6210e-01],
        [-6.3969e-01, -7.4963e-01]], device='

The out:  tensor([[-2.6189e-01, -1.4679e+00],
        [-3.0136e-03, -5.8061e+00],
        [-1.5520e-04, -8.7708e+00],
        ...,
        [-1.4406e+00, -2.7023e-01],
        [-5.8374e-01, -8.1601e-01],
        [-5.5855e-01, -8.4873e-01]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5934, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.3213, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0091,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0274, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0410, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-7.8089e-02, -2.5887e+00],
        [-1.5425e-03, -6.4751e+00],
        [-7.8198e-05, -9.4569e+00],
        ...,
        [-1.2071e+00, -3.5534e-01],
        [-5.3811e-01, -8.7672e-01],
        [-3.7604e-01, -1.1602e+00]], device='

10it [00:19,  1.96s/it]

The out:  tensor([[-4.8478e-01, -9.5667e-01],
        [-6.6032e-04, -7.3230e+00],
        [-5.8888e-05, -9.7395e+00],
        ...,
        [-9.7565e-01, -4.7312e-01],
        [-7.2268e-01, -6.6446e-01],
        [-5.9802e-01, -7.9829e-01]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.3160, 0.0000],
        [0.0000, 0.0775, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1003, 0.0000,  ..., 0.0000, 0.0471, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0102, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-1.5481e-02, -4.1759e+00],
        [-7.0094e-04, -7.2634e+00],
        [-1.6101e-03, -6.4323e+00],
        ...,
        [-9.5613e-01, -4.8512e-01],
        [-7.0761e-01, -6.7889e-01],
        [-4.1853e-01, -1.0730e+00]], device='

##### result in pandas dataframe

In [51]:
cv_result = pd.DataFrame([acc_list, sens_list, spec_list, mcc_list, auroc_list, auprc_list],
                         index=['Accuracy', 'Sensitivity', 'Specificity', 'MCC', 'AUROC', 'AUPRC'])
cv_result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Accuracy,0.865385,0.884615,0.903846,0.923077,0.923077,0.923077,0.884615,0.980392,0.921569,0.901961,0.865385,0.942308,0.903846,0.942308,0.884615,0.942308,0.865385,0.960784,0.941176,0.921569
Sensitivity,0.892857,0.892857,0.821429,0.888889,0.851852,0.888889,0.851852,0.962963,0.888889,0.925926,0.892857,0.928571,0.892857,0.888889,0.777778,0.925926,0.851852,0.962963,0.888889,0.925926
Specificity,0.833333,0.875000,1.000000,0.960000,1.000000,0.960000,0.920000,1.000000,0.958333,0.875000,0.833333,0.958333,0.916667,1.000000,1.000000,0.960000,0.880000,0.958333,1.000000,0.916667
MCC,0.728907,0.767857,0.824502,0.848889,0.856945,0.848889,0.771852,0.961480,0.845918,0.803409,0.728907,0.884932,0.807723,0.890871,0.791985,0.885270,0.731310,0.921296,0.888889,0.842593
AUROC,0.980655,0.947917,0.962054,0.983704,0.983704,0.977778,0.948148,0.993827,0.986111,0.984568,0.980655,0.947917,0.970982,0.988148,0.982222,0.974815,0.942222,0.993827,0.987654,0.978395
AUPRC,0.978370,0.964510,0.950674,0.982608,0.983256,0.976939,0.930333,0.992892,0.985534,0.984117,0.979032,0.963547,0.963597,0.985628,0.983054,0.971341,0.922398,0.992892,0.986366,0.979155


In [52]:
cv_result['Mean'] = cv_result.mean(axis=1)
cv_result['Mean']

Accuracy       0.914065
Sensitivity    0.890146
Specificity    0.940250
MCC            0.831621
AUROC          0.974765
AUPRC          0.972812
Name: Mean, dtype: float64

In [53]:
print('Cross-validation results:')
print('Stopping Epochs:', epoch_list)

Cross-validation results:
Stopping Epochs: [34, 30, 102, 36, 36, 83, 65, 46, 42, 98, 54, 59, 98, 41, 61, 56, 66, 42, 37, 92]


In [54]:
print(cv_result)

                    0         1         2         3         4         5  \
Accuracy     0.865385  0.884615  0.903846  0.923077  0.923077  0.923077   
Sensitivity  0.892857  0.892857  0.821429  0.888889  0.851852  0.888889   
Specificity  0.833333  0.875000  1.000000  0.960000  1.000000  0.960000   
MCC          0.728907  0.767857  0.824502  0.848889  0.856945  0.848889   
AUROC        0.980655  0.947917  0.962054  0.983704  0.983704  0.977778   
AUPRC        0.978370  0.964510  0.950674  0.982608  0.983256  0.976939   

                    6         7         8         9  ...        11        12  \
Accuracy     0.884615  0.980392  0.921569  0.901961  ...  0.942308  0.903846   
Sensitivity  0.851852  0.962963  0.888889  0.925926  ...  0.928571  0.892857   
Specificity  0.920000  1.000000  0.958333  0.875000  ...  0.958333  0.916667   
MCC          0.771852  0.961480  0.845918  0.803409  ...  0.884932  0.807723   
AUROC        0.948148  0.993827  0.986111  0.984568  ...  0.947917  0.9709

### Testing result

In [55]:
print('Testing progressing...')
model = Net(dim1, dim2, dim3, dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#optimizer = PCGrad(torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay))

Testing progressing...


In [56]:
model

Net(
  (conv1): RGCNConv(3226, 1340, num_relations=4)
  (lin1): Linear(in_features=1340, out_features=920, bias=True)
  (lin2): Linear(in_features=920, out_features=2, bias=True)
)

In [57]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.00195
    maximize: False
    weight_decay: 0.00738
)

In [58]:
auroc_val_max = 0
epoch_count = 0
for epoch in range(500):
    epoch_count += 1
    model_train(train_index)
    auroc_val, acc, sens, spec, mcc, auroc, auprc = model_val(val_index, plus_test=True)
    if auroc_val > auroc_val_max:
        epoch_count = 0
        auroc_val_max = auroc_val
        acc_max = acc
        auroc_max = auroc
        auprc_max = auprc
        sens_max = sens
        spec_max = spec
        mcc_max = mcc
        epoch_max = epoch
    if epoch_count == 30:
        break
        
test_result = pd.Series([acc_max, sens_max, spec_max, mcc_max, auroc_max, auprc_max],
                        index=['Accuracy', 'Sensitivity', 'Specificity', 'MCC', 'AUROC', 'AUPRC'])
print('Testing results at Epoch {}:'.format(epoch_max))

The out:  tensor([[-0.5072, -0.9217],
        [-1.1174, -0.3962],
        [-0.5891, -0.8093],
        ...,
        [-0.6261, -0.7651],
        [-1.2933, -0.3207],
        [-0.5974, -0.7991]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.4576, 0.0000, 0.7840,  ..., 0.0000, 0.2693, 0.7963],
        [0.8565, 0.5925, 0.2407,  ..., 0.6146, 1.7330, 0.3215],
        [1.3922, 1.1361, 0.0000,  ..., 0.7728, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0665, 0.5255, 0.5379],
        [1.9930, 0.5841, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5670, 0.5696, 0.0000,  ..., 1.0442, 0.0000, 0.0082]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-2.1093e+00, -1.2934e-01],
        [-1.6998e-04, -8.6801e+00],
        [-4.5662e-03, -5.3913e+00],
        ...,
        [-1.8505e+00, -1.7097e-01],
        [-2.5784e+00, -7.8932e-02],
        [-2.0317e+00, -1.4054e-01]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:

The out:  tensor([[-0.9595, -0.4830],
        [-0.1036, -2.3188],
        [-0.2928, -1.3710],
        ...,
        [-0.9259, -0.5045],
        [-1.0175, -0.4486],
        [-0.7349, -0.6531]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0',
       grad_fn=<ReluBackward0>)
The out:  tensor([[-1.2383, -0.3423],
        [-0.0678, -2.7254],
        [-0.0694, -2.7027],
        ...,
        [-0.7035, -0.6829],
        [-0.8244, -0.5771],
        [-0.6965, -0.6898]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..

The out:  tensor([[-1.0630, -0.4238],
        [-0.0463, -3.0953],
        [-0.0640, -2.7809],
        ...,
        [-0.6249, -0.7664],
        [-0.7485, -0.6407],
        [-0.3813, -1.1488]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.0411, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-0.8221, -0.5789],
        [-0.0469, -3.0829],
        [-0.0159, -4.1498],
        ...,
        [-0.5658, -0.8391],
        [-0.9685, -0.4774],
        [-0.4876, -0.9522]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 

The out:  tensor([[-3.2462e-01, -1.2830e+00],
        [-1.0133e-05, -1.1505e+01],
        [-1.4551e-03, -6.5334e+00],
        ...,
        [-4.1527e-01, -1.0793e+00],
        [-7.0395e-01, -6.8246e-01],
        [-2.6527e-01, -1.4567e+00]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)
The emb:  tensor([[0.0000, 0.2405, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)
The out:  tensor([[-2.2624e-01, -1.5971e+00],
        [-1.1844e-03, -6.7392e+00],
        [-8.3852e-04, -7.0843e+00],
        ...,
        [-5.7004e-01, -8.3356e-01],
        [-7.7752e-01, -6.1535e-01],
        [-2.6595e-01, -1.4545e+00]], device='

In [59]:
print(test_result)

Accuracy       0.900901
Sensitivity    0.881356
Specificity    0.923077
MCC            0.802864
AUROC          0.939700
AUPRC          0.907457
dtype: float64
